In [ ]:
# All subsequent cells summarized into one

SEGMENTS = ['ZUNU_vida-vessels.hdr', 'ZUNU_vida-sublobes.hdr', 'ZUNU_vida-lung.hdr', 'ZUNU_vida-airtree.hdr']
VidaDirectory = \
    r"Y:\E-Kelly-VidaSegmentals&Sublobes\3- 20Spiromics-10ForKelly-ForERS23\IA210305_H-17901_20151207_090246_20230524_121048_export"


# Find H-Number
import re
pattern = r'H-\d{5}'  # Regular expression pattern
matches = re.findall(pattern, VidaDirectory)  # Find all matches
HNUM = matches[0]  # Print the matched patterns
print('Found:', HNUM)


# Create VDysDir Dir
import os
VDysDir = os.path.join(VidaDirectory, 'VDysDir')
if not os.path.exists(VDysDir):
        os.makedirs(VDysDir)
print("VDysDir created in the VIDA folder")


# Search ImagesDirectory for H-NUM and find corresponding Case-#
ImagesDirectory = r"Y:\D-Images\SPIROMICS-SubStudy\2-Results-CheckedDoneTemp\1-Done"
def find_folder_with_hnum(path, HNUM):
    for root, dirs, files in os.walk(path):
        if root[-12:] == '1-Duplicates': continue
        print(root)
        for file in files:
            if HNUM in file:
                while True:
                    if "Case-" in os.path.basename(root):
                        return os.path.basename(root)
                    root = os.path.dirname(root)
                    if root == path or root == os.path.dirname(root):
                        break
    return None
result = find_folder_with_hnum(ImagesDirectory, HNUM)
if result: print("Found folder:", result)
else: print("No matching folder found.")
pattern = r'Case-\d{2}'  # Regular expression pattern
matches = re.findall(pattern, result)  # Find all matches
CNUM = matches[0]  # Print the matched patterns
print(CNUM+'\n')


# find all .nii.gz files, then pick the first! [print warning]
import glob
# Use glob to find all files with the extension ".nii.gz"
CASEPATH = os.path.join(ImagesDirectory, result)
IMGPATH = glob.glob(CASEPATH + '/**/*.nii.gz', recursive=True)
# Print the list of files
for IMPTH in IMGPATH:
    print(os.path.basename(IMPTH))
print('\nPicked the first as the target image')


# Copy image to VDysDir
import shutil
source_file = IMGPATH[0]
destination_folder = VDysDir
# Copy the file to the destination folder
shutil.copy(source_file, destination_folder)
print('Copied that image to VDysDir')


# List all files .nrrd, then find ones with the string 'label' > If several print warn > Pick last
# Use glob to find all files with the extension ".nrrd"
file_list = glob.glob(CASEPATH + '/**/*.nrrd', recursive=True)
# Filter the file list based on file names containing "Final" and "label"
LABELS = [file for file in file_list if 'Final' in file and 'label' in file]
if LABELS == []:
    print('WARNING: No Final-label.nrrd. FIX THAT PLEASE! Picking a non-label one.\n')
LABELS = [file for file in file_list if 'Final' in file]
for LABEL in LABELS:
    print(os.path.basename(LABEL))
LABELPATH = LABELS[-2]
print('\nPicked the last but one as the target labelmap')


# Copy nrrd labelmap to VDysDir
source_file = LABELPATH
destination_folder = VDysDir
# Copy the file to the destination folder
shutil.copy(source_file, destination_folder)
print('Copied that labelmap to VDysDir\n')


# List of segments = [..], Recenter them and put them in VDysDir
# re-center vessels and lung masks:
import SimpleITK as sitk
import numpy as np
imfn = os.path.join(VDysDir, os.path.basename(IMGPATH[0]))
iim = sitk.ReadImage(imfn)
for SEG in SEGMENTS:
    segfn = os.path.join(VidaDirectory, SEG)
    segim = sitk.ReadImage(segfn)
    snda = sitk.GetArrayFromImage(segim)
    snds = np.flip(snda, axis=0)
    fseg = sitk.GetImageFromArray(snds)
    fseg.CopyInformation(iim)
    outfn = SEG[0:-4] + '-recentered.nii.gz'
    sitk.WriteImage(fseg, os.path.join(VDysDir, outfn)) # flipped
    print('saved:', outfn)
print('\nDone.')

In [2]:
path = "Y:/public_resources/RA4-Project/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-15-MESAL-7018347-E3/MESAL-7018347-E3_FRC_80kV_0.75mm_Qr40d.nii.gz"
import SimpleITK as sitk
iim = sitk.ReadImage(path)
print(iim)


Image (000002425D712930)
  RTTI typeinfo:   class itk::Image<short,3>
  Reference Count: 1
  Modified Time: 2049
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 2024
  UpdateMTime: 2045
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 537]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 537]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 537]
  Spacing: [0.572266, 0.572266, 0.5]
  Origin: [-152.214, -316.714, -302.9]
  Direction: 
1 0 0
0 1 0
0 0 1

  IndexToPointMatrix: 
0.572266 0 0
0 0.572266 0
0 0 0.5

  PointToIndexMatrix: 
1.74744 0 0
0 1.74744 0
0 0 2

  Inverse Direction: 
1 0 0
0 1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (000002425F0C1A60)
      RTTI typeinfo:   class itk::ImportImageContainer<unsigned __i

In [15]:
#Get array of all directories with files to be copied
#Could add a file to flag directories that have already had their files copied over to allow this code to be repeatedly run
import pathlib
import os


dirPath = "Y:/public_resources/RA4-Project/D-Images/MESAL-3/2-Checked-Done-Temp/Done"

dirNamesList = []

#Stores all subdirectory names immediately within 'dirPath' - no recursion
for (dirpath, dirnames, filenames) in os.walk(dirPath):
    
    for currentDir in dirnames:
        #Ensures that the subdirectory is a completed 'Case' file before appending it to the array of directory names
        if(currentDir[:4] == 'Case-'):
            dirNamesList.append(str(currentDir))
        else:
            continue

print(dirNamesList)


#Generate array of all subdirectories in 'dirPath' using 'scandir' (highly efficient method)
#Stores all absolute paths to each subdirectory
pathList = [a.path for a in os.scandir(dirPath) if a.is_dir()]
#print(pathList)

i=0

useList = []

for dirs in pathList:
    #Ensures that the subdirectory is a completed 'Case' file before appending it to the array of paths to each subdirectory
    if(dirs[:4] == 'Case-'):
        useList.append(pathList[i])
    else:
        continue
    i+=1

#print(useList)




['Case-1-MESAL-5010098-E3', 'Case-10-MESAL-3016340-E3', 'Case-11-MESAL-3016587-E3', 'Case-12-MESAL-7018380-E3', 'Case-14-MESAL-6011020-E3', 'Case-15-MESAL-7018347-E3', 'Case-16-MESAL-5014832-E3', 'Case-17-MESAL-3012603-E3', 'Case-18-MESAL-6022812-E3', 'Case-19---MaskImageGeomMismatch---MESAL-3012840-E3', 'Case-2-MESAL-5010730-E3', 'Case-20-MESAL-7015569-E3', 'Case-21-MESAL-7018860-E3', 'Case-23---MaskImageGeomMismatch---MESAL-3015246-E3', 'Case-27-MESAL-6011594-E3', 'Case-3-MESAL-6011969-E3', 'Case-37-MESAL-3016900-E3', 'Case-4-MESAL-5010942-E3', 'Case-6-MESAL-7013450-E3', 'Case-7-MESAL-5010845-E3', 'Case-87-MESAL-7012705', 'Case-88-MESAL-6010377', 'Case-89-MESAL-4021096-E3', 'Case-9-MESAL-8014868-E3', 'Case-90-MESAL-5018196-E3', 'Case-91-MESAL-3021556-E3', 'Case-95-MESAL-4016645-E3', 'Markups', 'Vida_Masks', 'Adam', 'Duncan', 'Nate', 'Soheil', 'Markup_5010098', 'Markups', 'Vida_Masks', 'Adam', 'Jensen', 'Nate', 'Old Segmentation', 'Markups', 'Vida_Masks', 'Duncan', 'Jensen', 'Nate', '